# GPT Explainer

the purpose of this is to walk through what happens during the forward pass and backward pass of GPT-2 like models.  To help display the transformation, we'll use the first sentence from the [linear algebra wiki page](https://en.wikipedia.org/wiki/Linear_algebra) and [lu decomposition wiki page](https://en.wikipedia.org/wiki/LU_decomposition) as the topic is fitting and it shows us some non-standard patterns.  

## Text Prep/Tokenization

we'll start with common preprocessing step of tokenizing the data.  This converts the string text into an array of numbers that can be used during the training loop.  I've built a very subtle byte-pair encdoing that has each unique character that appears and the top 5 merges. This keeps our vocab size small and managable for this example. Typically the vocab size is in the 100K+ range. A great library for this is `tiktoken`. Tokenization simply finds the longest pattern of characters that's in common with what was trained and replaces it with an integer that represents it.  This way we turn the text into a numeric array to simplify computing. 

In [1]:
import torch
from collections import Counter

In [2]:
class SimpleBPETokenizer:
    def __init__(self, num_merges=5, eot_token='<|endoftext|>'):
        self.num_merges = num_merges
        self.eot_token = eot_token
        self.eot_id = None
        self.merges = []
        self.pair_ranks = {}
        self.vocab = {}
        self.id_to_token = {}

    def _add_token(self, tok):
        if tok in self.vocab:
            return self.vocab[tok]
        i = len(self.vocab)
        self.vocab[tok] = i
        self.id_to_token[i] = tok
        return i

    def _get_bigrams(self, seq):
        for i in range(len(seq) - 1):
            yield (seq[i], seq[i + 1])

    def _merge_once(self, seq, pair):
        a, b = pair
        out = []
        i = 0
        while i < len(seq):
            if i < len(seq) - 1 and seq[i] == a and seq[i + 1] == b:
                out.append(a + b)
                i += 2
            else:
                out.append(seq[i])
                i += 1
        return out

    def train(self, corpus):
        # corpus: list[str]
        text = ''.join(corpus).lower()
        seq = list(text)
        merges = []
        for _ in range(self.num_merges):
            counts = Counter(self._get_bigrams(seq))
            if not counts: break
            best_pair, _ = counts.most_common(1)[0]
            merges.append(best_pair)
            seq = self._merge_once(seq, best_pair)
        self.merges = merges
        self.pair_ranks = {p: i for i, p in enumerate(self.merges)}

        self.vocab = {}
        self.id_to_token = {}
        for ch in sorted(set(text)):
            self._add_token(ch)
        for a, b in self.merges:
            self._add_token(a + b)
        self.eot_id = self._add_token(self.eot_token)

    def encode(self, text, force_last_eot=True):
        # treat literal eot marker as special; remove it from content
        if self.eot_token in text:
            text = text.replace(self.eot_token, '')
        seq = list(text)

        # make sure all seen base chars exist
        for ch in set(seq):
            if ch not in self.vocab:
                self._add_token(ch)

        # greedy BPE using learned pair ranks
        if self.merges:
            while True:
                best_pair, best_rank = None, None
                for p in self._get_bigrams(seq):
                    r = self.pair_ranks.get(p)
                    if r is not None and (best_rank is None or r < best_rank):
                        best_pair, best_rank = p, r
                if best_pair is None:
                    break
                seq = self._merge_once(seq, best_pair)

        # ensure all tokens in seq exist in vocab (e.g., if new chars appeared)
        for tok in seq:
            if tok not in self.vocab:
                self._add_token(tok)

        ids = [self.vocab[tok] for tok in seq]

        # FORCE: append EOT id if not already last
        if force_last_eot:
            if not ids or ids[-1] != self.eot_id:
                ids.append(self.eot_id)

        return ids

    def decode(self, ids):
        # drop trailing EOT if present
        if ids and self.eot_id is not None and ids[-1] == self.eot_id:
            ids = ids[:-1]
        toks = [self.id_to_token[i] for i in ids]
        return ''.join(toks)


In [3]:
raw_example_1 = r'''Linear algebra is central to almost all areas of mathematics. For instance, linear algebra is fundamental in modern presentations of geometry, including for defining basic objects such as lines, planes and rotations. Also, functional analysis, a branch of mathematical analysis, may be viewed as the application of linear algebra to function spaces.'''
raw_example_2 = r'''In numerical analysis and linear algebra, lower–upper (LU) decomposition or factorization factors a matrix as the product of a lower triangular matrix and an upper triangular matrix (see matrix multiplication and matrix decomposition).'''


In [4]:
tok = SimpleBPETokenizer(num_merges=5)
tok.train([raw_example_1,raw_example_2])
tok.merges

[(' ', 'a'), ('a', 't'), ('i', 'n'), (' ', 'm'), ('i', 'o')]

In [5]:
tok.vocab

{' ': 0,
 '(': 1,
 ')': 2,
 ',': 3,
 '.': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g': 11,
 'h': 12,
 'i': 13,
 'j': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '–': 29,
 ' a': 30,
 'at': 31,
 'in': 32,
 ' m': 33,
 'io': 34,
 '<|endoftext|>': 35}

In [6]:
vocab_size = len(tok.vocab)
vocab_size

36

In [7]:
eot = tok.eot_id
tokens = []
for example in [raw_example_1, raw_example_2]:
    tokens.extend([eot])
    tokens.extend(tok.encode(example.lower()))
all_tokens = torch.tensor(tokens, dtype=torch.long)
all_tokens

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0,  7,
         9, 17, 22, 20,  5, 15,  0, 22, 18, 30, 15, 16, 18, 21, 22, 30, 15, 15,
        30, 20,  9,  5, 21,  0, 18, 10, 33, 31, 12,  9, 16, 31, 13,  7, 21,  4,
         0, 10, 18, 20,  0, 32, 21, 22,  5, 17,  7,  9,  3,  0, 15, 32,  9,  5,
        20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0, 10, 23, 17,  8,  5, 16,
         9, 17, 22,  5, 15,  0, 32, 33, 18,  8,  9, 20, 17,  0, 19, 20,  9, 21,
         9, 17, 22, 31, 34, 17, 21,  0, 18, 10,  0, 11,  9, 18, 16,  9, 22, 20,
        27,  3,  0, 32,  7, 15, 23,  8, 32, 11,  0, 10, 18, 20,  0,  8,  9, 10,
        32, 32, 11,  0,  6,  5, 21, 13,  7,  0, 18,  6, 14,  9,  7, 22, 21,  0,
        21, 23,  7, 12, 30, 21,  0, 15, 32,  9, 21,  3,  0, 19, 15,  5, 17,  9,
        21, 30, 17,  8,  0, 20, 18, 22, 31, 34, 17, 21,  4, 30, 15, 21, 18,  3,
         0, 10, 23, 17,  7, 22, 34, 17,  5, 15, 30, 17,  5, 15, 27, 21, 13, 21,
         3, 30,  0,  6, 20,  5, 17,  7, 

# Modeling

A machine learning model forward pass now uses the tokenization information, runs several layers of linear algebra on it, and then "predicts" the next token. When it is noisy (like you will see in this example), this process results in gibberish.  The training process changes the noise to pattern during the "backward pass" as you'll see.    We'll show 3 steps that are focused on training:
1. **Data Loading** `x, y = train_loader.next_batch()` - this step pulls from the raw data enough tokens to complete a forward and backward pass.  If the model is inference only, this step is replaced with taking in the inference input and preparing it similarly as the forward pass.
2. **Forward Pass** `logits, loss = model(x, y)` - using the data and the model architecture to predict the next token. When training we also compare against the expected to get loss, but in infrerence, we use the logits to complete the inference task.
3. **Backward Pass & Training** `loss.backward(); optimizer.step()` - using differentials to understand what parameters most impact the forward pass' impact on its prediction, comparing that against what is actually right based on the data loading step, and then making very minor adjustments to the impactful parameters with the hope it improves future predictions.

The we'll show a final **Forward Pass** with the updated weights we did in #3. 

## Data Loading

To start, we need to get enough data to run the forward and backward passes.  Since our total dataset is likely too big to hold all at once in real practice, we would read just enough file information into memory so that we can run the passes, leaving memory and compute to be used on the passes instead of static data holding. 
To start, we have to identify the batch size and the model context length to determine how much data we need.  Consequently, these dimensions also form 2 of the 3 dimensions in the initial matrix.
- **Batch Size (B)** - This is the number of examples you'll train on in a single pass. 
- **Context Length (T)** - This is the max number of tokens that a model can use in a single pass to generat the next token. If an example is below this length, it can be padded.
  
*Ideally both B and T are multiples of 2 to work nicely with chip architecture. This is a common theme across the board*

In [8]:
B = 2 # Batch
T = 8 # context length

To start, we need to pull from our long raw_token list enough tokens for the forward pass. To be able to satisfy training `B` Batches `T` Context length, we need to pull out `B*T` tokens to slide the context window across the examples enough to satisfy the batch size.  Since the training will attempt to predict the last token given the previous tokens in context, we also need 1 more token at the end so that the last training example in the last batch can have the next token to validate against. 

In [9]:
current_position = 0
tok_for_training = all_tokens[current_position:current_position + B*T +1 ]
tok_for_training

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0])

Now that we have our initial tokens to train on, we now need to convert it to a matrix that's ready for training. In this step we'll need to create our batches and setup two different arrays: 1/ the input, `x`, tokens that will result in 2/ the output `y` tokens. To create each example in the batch, every `T` tokens will be placed into it's own row. 

Recall that training takes in a string of tokens the length of the context and then predicts the next token. Recall that when we extracted `tok_for_training` we added 1 extra token so that we can evaluate the prediction for the last example. Because of this, the input, `x`, will be all of the tokens up to the second to last element `[:-1]`.  

It might be natural to think the output `y` would then just be the last token.But this is actually wasting valuable training loops.  Yes, there is the example that fills the context `T`, but we also have enough tokens in `tok_for_training` where any context length of `n` where `n<T` can also be used for inference since we have the `n+1` token available.  You can think of the following example:

sentence: `Hi I am learning`. This sentence contains the following "next tokens" that can be learned:
1. x: Hi I am  | y: learning
2. x: Hi I     | y: am
3. x: Hi       | y: I

Because we have this triangle to create, our `y` can be much larger.  We can start with the second token and, go all the way to the last element we added for the last example `[1:'`.   


We will now put this together and do the following:
1. Extract the input `x` and then split it into an example for each batch `B`
2. Extract the output `y` and then split it into an example for each batch `B`

*Note: View can take `-1` which allows the matrix to infer the dimension so we do not need to pass in `T`, but given how many matrices we'll work with we want to make sure we're controlling the dimensions or erroring out if they do not match our expectations.*

In [10]:
tok_for_training

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0])

In [11]:
x=tok_for_training[:-1].view(B, T)
x

tensor([[35, 15, 32,  9,  5, 20, 30, 15],
        [11,  9,  6, 20,  5,  0, 13, 21]])

In [12]:
y=tok_for_training[1:].view(B, T)
y

tensor([[15, 32,  9,  5, 20, 30, 15, 11],
        [ 9,  6, 20,  5,  0, 13, 21,  0]])

## Forward pass

<img src="explainer_screenshots/full_network.png" width="200">

The forward pass takes a string of tokens in and predicts the next "n" tokens.  This step as we'll look at it is focused on training where we'll pass in the input `x`, carry that input through the layers, and generate a matrix of the probability of each token being the next one, something we call `logits`. During the forward pass at the end we then compare the probability to the actual next token in `y` and calculate `loss` based on the difference. 

*Note that we will do some layer initialization to simplify following along.  In reality layers are often initialized to normal distribution with some adjustments made for parameter sizes to keep the weights properly noisy.  We will not cover initialization in this series*

We first rederive the batch size and context size based on the input to improve flexibility.

In [13]:
import torch.nn as nn

In [14]:
B, T = x.size()
B,T

(2, 8)

The first layer of our network creates an embedding representation of our input sequence. 

In [15]:
block_size = 8 # max sequence length/context
vocab_size = vocab_size # 36 
n_embd = 4 # 

### Input Layer

<img src="explainer_screenshots/input_layer.png" width="200">
We'll first create an initiation for 2 of our input matrices: position and token embedding.  Both of these are a table of weigths that have `n_embd` number of columns to store information about the position or token. The more columns you add, the more complex information can be stored but the more compute is needed.  For now we'll let each position or column store up to 4 channels of information.  Before starting though we need to initialize the layer with a set of data.

In [16]:
# weighted token embedding
wte = nn.Embedding(vocab_size, n_embd)
with torch.no_grad(): # initilize to W[i,j] = 0.001*(1+i+j) for easy following 
    vs, d = wte.num_embeddings, wte.embedding_dim
    rows = torch.arange(vs).unsqueeze(1)  # (vs,1)
    cols = torch.arange(d).unsqueeze(0)  # (1,d)
    pattern = 0.01*(1 + rows + cols)  # W[i,j] = 0.001*(1+i+j)
    wte.weight.copy_(pattern)
# weighted position embedding
wpe = nn.Embedding(block_size, n_embd)
with torch.no_grad(): # initilize to W[i,j] = 0.001*(1+i+j) for easy following 
    vs, d = wpe.num_embeddings, wpe.embedding_dim
    rows = torch.arange(vs).unsqueeze(1)  # (vs,1)
    cols = torch.arange(d).unsqueeze(0)  # (1,d)
    pattern = 0.01*(1 + rows + cols)  # W[i,j] = 0.001*(1+i+j)
    wpe.weight.copy_(pattern)
wte.weight, wpe.weight

(Parameter containing:
 tensor([[0.0100, 0.0200, 0.0300, 0.0400],
         [0.0200, 0.0300, 0.0400, 0.0500],
         [0.0300, 0.0400, 0.0500, 0.0600],
         [0.0400, 0.0500, 0.0600, 0.0700],
         [0.0500, 0.0600, 0.0700, 0.0800],
         [0.0600, 0.0700, 0.0800, 0.0900],
         [0.0700, 0.0800, 0.0900, 0.1000],
         [0.0800, 0.0900, 0.1000, 0.1100],
         [0.0900, 0.1000, 0.1100, 0.1200],
         [0.1000, 0.1100, 0.1200, 0.1300],
         [0.1100, 0.1200, 0.1300, 0.1400],
         [0.1200, 0.1300, 0.1400, 0.1500],
         [0.1300, 0.1400, 0.1500, 0.1600],
         [0.1400, 0.1500, 0.1600, 0.1700],
         [0.1500, 0.1600, 0.1700, 0.1800],
         [0.1600, 0.1700, 0.1800, 0.1900],
         [0.1700, 0.1800, 0.1900, 0.2000],
         [0.1800, 0.1900, 0.2000, 0.2100],
         [0.1900, 0.2000, 0.2100, 0.2200],
         [0.2000, 0.2100, 0.2200, 0.2300],
         [0.2100, 0.2200, 0.2300, 0.2400],
         [0.2200, 0.2300, 0.2400, 0.2500],
         [0.2300, 0.2400, 0.250

**Positional Embeddings** - Now we need to pluck the weight of each position out of the position embedding.  Since we are creating a simple left to right, position 1 to n, we can just create an array from 0 to n based on the context, `T`, then pluck those rows out.  The resulting matrix from this operation is a `T, n_embd` based vector. 

In [17]:
pos = torch.arange(0, T, dtype=torch.long)
pos

tensor([0, 1, 2, 3, 4, 5, 6, 7])

for each element, look up the row in `wpe` and pluck it out. Since the position is just `[0:T]` we can see we pluck out the position array.

In [18]:
pos_emb = wpe(pos)
pos_emb.shape, pos_emb

(torch.Size([8, 4]),
 tensor([[0.0100, 0.0200, 0.0300, 0.0400],
         [0.0200, 0.0300, 0.0400, 0.0500],
         [0.0300, 0.0400, 0.0500, 0.0600],
         [0.0400, 0.0500, 0.0600, 0.0700],
         [0.0500, 0.0600, 0.0700, 0.0800],
         [0.0600, 0.0700, 0.0800, 0.0900],
         [0.0700, 0.0800, 0.0900, 0.1000],
         [0.0800, 0.0900, 0.1000, 0.1100]], grad_fn=<EmbeddingBackward0>))

**Word Embeddings** - Similarly we need to pluck out the rows from the token table, `wte` for the tokens in our example. Since our example is already represented as indices, we can simple use `x` directly. The resulting matrix from this operation is a `B,T, n_embd` based vector since `x` is `B,T` and `tok_emb` is `vocab_size,n_embd` and when we index `wte` by `x` each entry in x replaces `n_embd` based vector at that position in `wte`.

In [19]:
x

tensor([[35, 15, 32,  9,  5, 20, 30, 15],
        [11,  9,  6, 20,  5,  0, 13, 21]])

for each position pull out the the row of weights that corresponds to the token. You can see in the print out that the rows are not in the same order as the layer is initiliazed as the token ids are not sequential.  

In [20]:
tok_emb = wte(x)
tok_emb.shape, tok_emb

(torch.Size([2, 8, 4]),
 tensor([[[0.3600, 0.3700, 0.3800, 0.3900],
          [0.1600, 0.1700, 0.1800, 0.1900],
          [0.3300, 0.3400, 0.3500, 0.3600],
          [0.1000, 0.1100, 0.1200, 0.1300],
          [0.0600, 0.0700, 0.0800, 0.0900],
          [0.2100, 0.2200, 0.2300, 0.2400],
          [0.3100, 0.3200, 0.3300, 0.3400],
          [0.1600, 0.1700, 0.1800, 0.1900]],
 
         [[0.1200, 0.1300, 0.1400, 0.1500],
          [0.1000, 0.1100, 0.1200, 0.1300],
          [0.0700, 0.0800, 0.0900, 0.1000],
          [0.2100, 0.2200, 0.2300, 0.2400],
          [0.0600, 0.0700, 0.0800, 0.0900],
          [0.0100, 0.0200, 0.0300, 0.0400],
          [0.1400, 0.1500, 0.1600, 0.1700],
          [0.2200, 0.2300, 0.2400, 0.2500]]], grad_fn=<EmbeddingBackward0>))

**Impact of the position and token together**
To ensure that the position and token together impact the next token prediction, we sum the two so that the weight of each token is impacted by the weight of its relative position. To do this we sum `tok_emb` and `pos_emb` together. Quickly we can see the dimensions don't match as 
* `tok_emb` > `B,T,n_embd`
* `pos_emb` >   `T,n_embd`

Since we have multiple examples with the same ordering, we simply add pos_emb at the same level to each entry on the `B` dimension, something that pytorch does for us automatically resulting in a `B,T,n_embd` output

In [21]:
x = tok_emb + pos_emb
x

tensor([[[0.3700, 0.3900, 0.4100, 0.4300],
         [0.1800, 0.2000, 0.2200, 0.2400],
         [0.3600, 0.3800, 0.4000, 0.4200],
         [0.1400, 0.1600, 0.1800, 0.2000],
         [0.1100, 0.1300, 0.1500, 0.1700],
         [0.2700, 0.2900, 0.3100, 0.3300],
         [0.3800, 0.4000, 0.4200, 0.4400],
         [0.2400, 0.2600, 0.2800, 0.3000]],

        [[0.1300, 0.1500, 0.1700, 0.1900],
         [0.1200, 0.1400, 0.1600, 0.1800],
         [0.1000, 0.1200, 0.1400, 0.1600],
         [0.2500, 0.2700, 0.2900, 0.3100],
         [0.1100, 0.1300, 0.1500, 0.1700],
         [0.0700, 0.0900, 0.1100, 0.1300],
         [0.2100, 0.2300, 0.2500, 0.2700],
         [0.3000, 0.3200, 0.3400, 0.3600]]], grad_fn=<AddBackward0>)

### Transformer Layers

<img src="explainer_screenshots/transformer.png" width="200">

The transformer block is multiple parallel repetitions of the same matrix operations done independently.  This adds both depth and breadth to the computation.  Each block is the same steps of
1. Layer normalization
2. Causal self attention
3. Layer normalization (again)
4. Multi-layer perceptron (MLP)

Both steps 2 and 4 are also multi-layered so we'll go through each layer independently.  You'll notice the arrows in the diagram bypassing the causal self attention and the MLP.  This is to ensure that the weights of any one layer do not get overweighted. We achieve this by simply adding the input with the layer's calculations together, as you'll see. 

#### Transformer - Layer Normalization

With Layer normalization, we review the row and adjust based on how far away it is from the mean. This means an array of `[1,2,3,4]` and `[2,4,6,8]` will actually have the same normalized entries after layer normalization.  This layer adds regularization which helps with overall learning speed. The formula applied is:

$y = \frac{x - \mathbb{E}[x]}{\sqrt{\operatorname{Var}[x] + \epsilon}}$

Layer normalization is applied on the input matrix in kind and creates default weights of 1's across the dimension to equally weight all values in the normalization. We'll keep this as is and not change the initialization.  

*Note that even though we will do layer normalization again, we keep this as a separate layer so that its impact can be adjusted independent of other normalization layers.*

**Example** Let's see a quick example of how layer normalizaiton operates with an array of `[1,2,3,4]` and `[2,4,6,8]` 

In [22]:
example_ln = nn.LayerNorm(n_embd)

## Example 
example_ln(torch.tensor(
    [
        [1.0,2.0,3.0,4.0],
        [2.0,4.0,6.0,8.0]
    ]))

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]],
       grad_fn=<NativeLayerNormBackward0>)

**Normalize** Now let's apply it to x.  We'll save the output to a new variable `x_norm` so that we retain the input `x` for the *skip connection* (more details below).  Since our position embeddings are still in their initiaion step where each entries values are similarly distributed `[n,n+2,n+4,n+6]`  we'll see that with layer normalization the resulting matrix has all equal rows. 

In [23]:
ln_1 = nn.LayerNorm(n_embd)
x_norm = ln_1(x)
x_norm.shape, x_norm

(torch.Size([2, 8, 4]),
 tensor([[[-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284]],
 
         [[-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284]]],
        grad_fn=<NativeLayerNormBackward0>))

#### Transformer - Flash Attnetion / Causal Self Attention

<img src="explainer_screenshots/flash_attention.png" width="600">

Causal self-attention is self-attention with a strictly lower-triangular mask, so each position $i$ can attend only to positions $j≤i$ (no look-ahead). For each position in a sequence, the model decides which earlier tokens matter and blends their information to generate the upcoming token, but the model is not allowed to peek at tokens to the right. Recall that we created an input `x` that has all of the tokens in the sequence. With causal self attention we ensure that the prediction of `y` is not learned in an example from tokens to the right of it. 

Causal self attention uses 3 linear projections to create representations of 3 concepts to help derive the next token: query, key, and value. FlashAttention takes the input `x` of shape $(B,T,C)$, linearly projects the last dimension to `Q, K, V` by multiplying each token vector $x_{b,t,:}$ with learned weight matrices `Q, K,V` (often via one combined QKV linear) and adding biases (which we do not have. Then we compute the causal dot-product attention with softmax per head, concatenate the heads, and apply a final linear layer to produce the output.  Linear layers use the projection of $y=xA^\top+b$

The Q,K,V matrix do the following: 
* **Query** - the current position’s “search request.” Every layer and head issues queries that can look for many things (antecedent, rhythm, agreement, etc.).
* **Key** - a matching tag/address for each allowed position. It is compared with the query to produce relevance scores
* **Value** - the payload you actually mix in once something matches. It’s a learned projection of the token’s representation so the model can copy the right kind of information.

This layer starts by taking a linear layer that has `n_embd` rows and then  `n_embd` columns for each: the query, key and value.  This allows the projection of the `x_norm` to be split into the 3 components, selected from them. We then use `scaled_dot_product_attention` to provide the attention masking and normalization, including a lower triangle mask to prevent look-ahead, and final project that onto a `n_embd x n_embd` matrix to return the output. 

We start by creating the 2 default layers. Also we'll set heads to 2.  The heads allow the layer to specialize in different concepts since each head creates its own Q,K,V.

*for the linear layers we'll do some special initiations, mainly pyramidal so that we can see unique numbers. We also initialize the bias to 0 so there linear layer is now just $y=xA^\top$*

In [24]:
from torch.nn import functional as F
n_head = 2

In [25]:
c_attn = nn.Linear(n_embd, 3 * n_embd)
with torch.no_grad():
    out, inp = c_attn.weight.shape  # (3*n_embd, n_embd)
    r = torch.arange(1, out + 1).unsqueeze(1)  # [out,1], 1-indexed
    c = torch.arange(1, inp + 1).unsqueeze(0)   # [1,inp], 1-indexed
    base = r * c                          # rc
    tri = r * (r - 1) / 2                 # T_{r-1} = r(r-1)/2, shape [out,1]
    mask = (c >= 2)           # add T_{r-1} only from column 2 onward
    pattern = 1e-3 * (base + tri * mask) # matches [[.001,.002,.003],[.002,.005,.007],[.003,.009,.012],...]
    c_attn.weight.copy_(pattern)

    c_attn.bias.zero_()
    
c_attn, c_attn.weight, c_attn.bias

(Linear(in_features=4, out_features=12, bias=True),
 Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040],
         [0.0020, 0.0050, 0.0070, 0.0090],
         [0.0030, 0.0090, 0.0120, 0.0150],
         [0.0040, 0.0140, 0.0180, 0.0220],
         [0.0050, 0.0200, 0.0250, 0.0300],
         [0.0060, 0.0270, 0.0330, 0.0390],
         [0.0070, 0.0350, 0.0420, 0.0490],
         [0.0080, 0.0440, 0.0520, 0.0600],
         [0.0090, 0.0540, 0.0630, 0.0720],
         [0.0100, 0.0650, 0.0750, 0.0850],
         [0.0110, 0.0770, 0.0880, 0.0990],
         [0.0120, 0.0900, 0.1020, 0.1140]], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True))

In [26]:
c_proj = nn.Linear(n_embd, n_embd)
with torch.no_grad():
    out, inp = c_proj.weight.shape  # (3*n_embd, n_embd)
    r = torch.arange(1, out + 1).unsqueeze(1)  # [out,1], 1-indexed
    c = torch.arange(1, inp + 1).unsqueeze(0)   # [1,inp], 1-indexed
    base = r * c                          # rc
    tri = r * (r - 1) / 2                 # T_{r-1} = r(r-1)/2, shape [out,1]
    mask = (c >= 2)           # add T_{r-1} only from column 2 onward
    pattern = 1e-3 * (base + tri * mask) # matches [[.001,.002,.003],[.002,.005,.007],[.003,.009,.012],...]
    c_proj.weight.copy_(pattern)
    
    c_proj.bias.zero_()
c_proj, c_proj.weight

(Linear(in_features=4, out_features=4, bias=True),
 Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040],
         [0.0020, 0.0050, 0.0070, 0.0090],
         [0.0030, 0.0090, 0.0120, 0.0150],
         [0.0040, 0.0140, 0.0180, 0.0220]], requires_grad=True))

**Flash Attention - Creating Query, Key, Value**

We'll now create the query, key, and value matrices. Let's also revisualize `x_norm` to make it easy to connect the dots in this complex layer.

In [27]:
B, T, C = x_norm.size()
B, T, C

(2, 8, 4)

In [28]:
x_norm.shape, x_norm

(torch.Size([2, 8, 4]),
 tensor([[[-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284]],
 
         [[-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284],
          [-1.3284, -0.4428,  0.4428,  1.3284]]],
        grad_fn=<NativeLayerNormBackward0>))

Now we'll take `x_norm` and the dot product of the weights in `c_attn` $y=x\_norm \cdot c\_attn^\top+bias$. This results in the the `qkv` combined matrix leading to a 3x size increase. Since every entry on the 3rd dimension in `x_norm` is the same, we will see that every row has the same repeated values given that's how dot products work. 

In [29]:
qkv = c_attn(x_norm)
qkv.shape, qkv

(torch.Size([2, 8, 12]),
 tensor([[[0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408],
          [0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408],
          [0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408],
          [0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408],
          [0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408],
          [0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408],
          [0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408],
          [0.0044, 0.0102, 0.0173, 0.0257, 0.0354, 0.0465, 0.0589, 0.0726,
           0.0877, 0.1041, 0.1218, 0.1408]],
 
         [[0.0044, 0

**Flash Attention - Attention Head**

Now, we will split up qkv to create the 3 separate matrices, one for the query, key, and value. Together they to work to create complex concept embeddings. We also then have to split up each matrix into its own heads (shown as columns `dim=3`). For each head we create its own Q,K,V, compute the causal dot-product attention with softmax per head and then eventually concatenate the heads. During back-propogation, different heads can get updates in different ways based on the examples and gradients.  This backprop and separation is what allows each attention head to specialize in different concepts.

In [30]:
q,k,v = qkv.split(n_embd, dim=2)
q = q.view(B, T, n_head, C // n_head).transpose(1, 2)
k = k.view(B, T, n_head, C // n_head).transpose(1, 2)
v = v.view(B, T, n_head, C // n_head).transpose(1, 2)
'q', q.shape, q, 'k', k.shape, k, 'v', v.shape, v

('q',
 torch.Size([2, 2, 8, 2]),
 tensor([[[[0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102]],
 
          [[0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257]]],
 
 
         [[[0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102],
           [0.0044, 0.0102]],
 
          [[0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257],
           [0.0173, 0.0257]]]], grad_fn=<TransposeBackwa

**Flash Attention - Cross Attention / Dot-product attention**

With the separated query, key, value, we now need to calculate the dot product, but use a lower triangle to restrict each position to only be able to use embeddings for tokens that precede it.  To do this, flash attention tiles Q,K,V and apply masking and softmax on the fly. If done without flash attention, you'd need to materialize the large `(T,T)` matrix for all the queries and keys, then apply a lower triangle mask fill to zero out the upper triangle to prevent "look ahead", compute the dot product and finally apply a softmax.  After running this calculation per head we collapse the heads together back into the same dimension. 

In [31]:
fa = F.scaled_dot_product_attention(q, k, v, is_causal=True)
fa, fa.shape

(tensor([[[[0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041]],
 
          [[0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408]]],
 
 
         [[[0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041],
           [0.0877, 0.1041]],
 
          [[0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408],
           [0.1218, 0.1408]]]],
        grad_fn=<ScaledDotProductFlashAttentionForCpuBack

**Flash Attention - collapse heads**  Now we'll reshape back to remove the heads

*Note we use `contiguous` here to force `transpose()` to create a new matrix in memory. This allows the heads to learn independently*

In [32]:
fa = fa.transpose(1, 2).contiguous().view(B, T, C)
fa.shape, fa

(torch.Size([2, 8, 4]),
 tensor([[[0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408]],
 
         [[0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408],
          [0.0877, 0.1041, 0.1218, 0.1408]]], grad_fn=<ViewBackward0>))

**Flash Attention - final projection and output**
Finally, we will now project the cross attention matrix on another final linear layer `n_embd x n_embd` that we initialized as `c_proj`.  Once agian, because our rows are identical and we use `c_attn` $y=x\_norm \cdot c\_proj^\top$ we will result in the same values. Note the removal of bias since we set it to 0.

In [33]:
x_norm = c_proj(fa)
x_norm.shape, x_norm

(torch.Size([2, 8, 4]),
 tensor([[[0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071]],
 
         [[0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071],
          [0.0012, 0.0028, 0.0048, 0.0071]]], grad_fn=<ViewBackward0>))

#### Transformer - Residual (skip) connection

<img src="explainer_screenshots/skip_layer.png" width="200">
Modern networks also use skip connections, meaning they allow for pathways to bypass around "boxes", passing through gradients during the backward pass.  This attribute ensures that the impact of each layer and head is normalized against the input embeddings themselves. Recall in the diagram we had the arrow that bypassed "masked multiheaded attention".  Functionally this is represented as

$y = f(x) + x$

To achieve this we simply sum the projection matrix `x` with the flash attention output `x_norm`.  As a reminder we'll print out X.  As you can see, because `x` was based on the tokens, it has a different value per row, so even though `x_norm` has the same value per row, we'll result in a diverse set of weights.  With this you can see the power of skip connections passing through weights.

In [34]:
x

tensor([[[0.3700, 0.3900, 0.4100, 0.4300],
         [0.1800, 0.2000, 0.2200, 0.2400],
         [0.3600, 0.3800, 0.4000, 0.4200],
         [0.1400, 0.1600, 0.1800, 0.2000],
         [0.1100, 0.1300, 0.1500, 0.1700],
         [0.2700, 0.2900, 0.3100, 0.3300],
         [0.3800, 0.4000, 0.4200, 0.4400],
         [0.2400, 0.2600, 0.2800, 0.3000]],

        [[0.1300, 0.1500, 0.1700, 0.1900],
         [0.1200, 0.1400, 0.1600, 0.1800],
         [0.1000, 0.1200, 0.1400, 0.1600],
         [0.2500, 0.2700, 0.2900, 0.3100],
         [0.1100, 0.1300, 0.1500, 0.1700],
         [0.0700, 0.0900, 0.1100, 0.1300],
         [0.2100, 0.2300, 0.2500, 0.2700],
         [0.3000, 0.3200, 0.3400, 0.3600]]], grad_fn=<AddBackward0>)

In [35]:
x = x + x_norm
x.shape, x

(torch.Size([2, 8, 4]),
 tensor([[[0.3712, 0.3928, 0.4148, 0.4371],
          [0.1812, 0.2028, 0.2248, 0.2471],
          [0.3612, 0.3828, 0.4048, 0.4271],
          [0.1412, 0.1628, 0.1848, 0.2071],
          [0.1112, 0.1328, 0.1548, 0.1771],
          [0.2712, 0.2928, 0.3148, 0.3371],
          [0.3812, 0.4028, 0.4248, 0.4471],
          [0.2412, 0.2628, 0.2848, 0.3071]],
 
         [[0.1312, 0.1528, 0.1748, 0.1971],
          [0.1212, 0.1428, 0.1648, 0.1871],
          [0.1012, 0.1228, 0.1448, 0.1671],
          [0.2512, 0.2728, 0.2948, 0.3171],
          [0.1112, 0.1328, 0.1548, 0.1771],
          [0.0712, 0.0928, 0.1148, 0.1371],
          [0.2112, 0.2328, 0.2548, 0.2771],
          [0.3012, 0.3228, 0.3448, 0.3671]]], grad_fn=<AddBackward0>))

#### Transformer - Layer Normalization 2

We'll run another round of normalization now on the outputs of the masked multi-head attention and skip connection to ensure our values are not too spread apart. This layer will run the same normalization formula as before, but is it's own independent layer as it has different inputs. Recall the formula is: 

$y = \frac{x - \mathbb{E}[x]}{\sqrt{\operatorname{Var}[x] + \epsilon}}\$

Because we will do a skip connection again for the next layer, MLP, we'll once again branch `x` for the normalization and MLP and then sum it back togehter with `x`. 

Similar to before you'll notice that normalization brings all the values back to the same entry.  Despite `x` having different values, the distribution of the values is the same which is why normalization brings them back. 

In [36]:
ln_2 = nn.LayerNorm(n_embd)

In [37]:
x_norm_2 = ln_2(x)
x_norm_2.shape, x_norm_2

(torch.Size([2, 8, 4]),
 tensor([[[-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380]],
 
         [[-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380],
          [-1.3232, -0.4509,  0.4361,  1.3380]]],
        grad_fn=<NativeLayerNormBackward0>))

#### Transformer - Feed Forward (aka Multi-layer Perceptron)

<img src="explainer_screenshots/mlp.png" width="400">

The feed-forward sublayer consists of two-layer mirroring a multi-layer perceptron, MLP.  These layers mix the features within each token vector but never across time. The output `x` from multi-headed attention starts as `B x T x C`.  Feed forward 
1. Calculates `4C` using a $XA^\top + B$ linear layer 
2. Normalizes the data using a `tanh` based GELU layer. This layer pushes extreme values to +/- 1
3. Projects back down to `C` with a final $XW^\top + B$ linear layer.

The MLP nonlinearly re-expresses each token's channel features before being aggregated across the hidden layers and passed to the output layer. 

We'll first start by creating the 3 different layers:
1. `mlp_fc` - Linear layer to project up to `4C`
2. `mlp_gelu` - tanh layer
3. `mlp_proj` - Linear layer to project down to `C`

We'll do similar initiation to our linear layers as before. Since the tanh step is a calculation per row (similar to layer normalization), we will not do initiation for it.

In [38]:
mlp_fc = nn.Linear(n_embd, 4 * n_embd)

with torch.no_grad():
    out, inp = mlp_fc.weight.shape  # (3*n_embd, n_embd)
    r = torch.arange(1, out + 1).unsqueeze(1)  # [out,1], 1-indexed
    c = torch.arange(1, inp + 1).unsqueeze(0)   # [1,inp], 1-indexed
    base = r * c                          # rc
    tri = r * (r - 1) / 2                 # T_{r-1} = r(r-1)/2, shape [out,1]
    mask = (c >= 2)           # add T_{r-1} only from column 2 onward
    pattern = 1e-3 * (base + tri * mask) # matches [[.001,.002,.003],[.002,.005,.007],[.003,.009,.012],...]
    mlp_fc.weight.copy_(pattern)

mlp_fc.weight.shape, mlp_fc.weight

(torch.Size([16, 4]),
 Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040],
         [0.0020, 0.0050, 0.0070, 0.0090],
         [0.0030, 0.0090, 0.0120, 0.0150],
         [0.0040, 0.0140, 0.0180, 0.0220],
         [0.0050, 0.0200, 0.0250, 0.0300],
         [0.0060, 0.0270, 0.0330, 0.0390],
         [0.0070, 0.0350, 0.0420, 0.0490],
         [0.0080, 0.0440, 0.0520, 0.0600],
         [0.0090, 0.0540, 0.0630, 0.0720],
         [0.0100, 0.0650, 0.0750, 0.0850],
         [0.0110, 0.0770, 0.0880, 0.0990],
         [0.0120, 0.0900, 0.1020, 0.1140],
         [0.0130, 0.1040, 0.1170, 0.1300],
         [0.0140, 0.1190, 0.1330, 0.1470],
         [0.0150, 0.1350, 0.1500, 0.1650],
         [0.0160, 0.1520, 0.1680, 0.1840]], requires_grad=True))

In [39]:
mlp_gelu = nn.GELU(approximate='tanh')
mlp_gelu

GELU(approximate='tanh')

In [40]:
mlp_proj = nn.Linear(4 * n_embd, n_embd)

with torch.no_grad():
    out, inp = mlp_proj.weight.shape  # (3*n_embd, n_embd)
    r = torch.arange(1, out + 1).unsqueeze(1)  # [out,1], 1-indexed
    c = torch.arange(1, inp + 1).unsqueeze(0)   # [1,inp], 1-indexed
    base = r * c                          # rc
    tri = r * (r - 1) / 2                 # T_{r-1} = r(r-1)/2, shape [out,1]
    mask = (c >= 2)           # add T_{r-1} only from column 2 onward
    pattern = 1e-3 * (base + tri * mask) # matches [[.001,.002,.003],[.002,.005,.007],[.003,.009,.012],...]
    mlp_proj.weight.copy_(pattern)

mlp_proj.weight.shape, mlp_proj.weight

(torch.Size([4, 16]),
 Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040, 0.0050, 0.0060, 0.0070, 0.0080, 0.0090,
          0.0100, 0.0110, 0.0120, 0.0130, 0.0140, 0.0150, 0.0160],
         [0.0020, 0.0050, 0.0070, 0.0090, 0.0110, 0.0130, 0.0150, 0.0170, 0.0190,
          0.0210, 0.0230, 0.0250, 0.0270, 0.0290, 0.0310, 0.0330],
         [0.0030, 0.0090, 0.0120, 0.0150, 0.0180, 0.0210, 0.0240, 0.0270, 0.0300,
          0.0330, 0.0360, 0.0390, 0.0420, 0.0450, 0.0480, 0.0510],
         [0.0040, 0.0140, 0.0180, 0.0220, 0.0260, 0.0300, 0.0340, 0.0380, 0.0420,
          0.0460, 0.0500, 0.0540, 0.0580, 0.0620, 0.0660, 0.0700]],
        requires_grad=True))

**MLP - 4c projection**  - Now we'll take `x_norm_2` and apply the first linear layer that projects upward to `4C`.  Reminder that the  calculation is $x\_mlp = x\_norm\_2 \cdot mlp\_fc^\top$. Once again we'll see a repetition of values in `x_mlp` as the rows in `x_norm_2` are the same.

In [41]:
x_mlp = mlp_fc(x_norm_2)
x_mlp.shape, x_mlp

(torch.Size([2, 8, 16]),
 tensor([[[-0.1647, -0.3401, -0.0664, -0.4235, -0.3288, -0.3806,  0.2099,
           -0.1826,  0.0884,  0.5907, -0.1220, -0.2782,  0.4654,  0.6769,
           -0.2593, -0.0073],
          [-0.1647, -0.3401, -0.0664, -0.4235, -0.3288, -0.3806,  0.2099,
           -0.1826,  0.0884,  0.5907, -0.1220, -0.2782,  0.4654,  0.6769,
           -0.2593, -0.0073],
          [-0.1647, -0.3401, -0.0664, -0.4235, -0.3288, -0.3806,  0.2099,
           -0.1826,  0.0884,  0.5907, -0.1220, -0.2782,  0.4654,  0.6768,
           -0.2593, -0.0073],
          [-0.1647, -0.3401, -0.0664, -0.4235, -0.3288, -0.3806,  0.2099,
           -0.1826,  0.0884,  0.5907, -0.1220, -0.2782,  0.4654,  0.6769,
           -0.2593, -0.0073],
          [-0.1647, -0.3401, -0.0664, -0.4235, -0.3288, -0.3806,  0.2099,
           -0.1826,  0.0884,  0.5907, -0.1220, -0.2782,  0.4654,  0.6769,
           -0.2593, -0.0073],
          [-0.1647, -0.3401, -0.0664, -0.4235, -0.3288, -0.3806,  0.2099,
           

**MLP - tanh**  - Now we'll apply the tanh approximation (GELU - tanh) which smoothly gates each input `x_mlp`. The formula applied is 

$\tanh(x)=\frac{\exp(x)-\exp(-x)}{\exp(x)+\exp(-x)}$

The Tanh formula pushes large positive numbers to 1 and large negative numbers to -1.  `tanh` is applied element wise across the full `x_mlp`

In [42]:
x_mlp = mlp_gelu(x_mlp)
x_mlp.shape, x_mlp

(torch.Size([2, 8, 16]),
 tensor([[[-0.0716, -0.1248, -0.0315, -0.1423, -0.1220, -0.1339,  0.1224,
           -0.0781,  0.0473,  0.4268, -0.0551, -0.1086,  0.3161,  0.5081,
           -0.1031, -0.0036],
          [-0.0716, -0.1248, -0.0315, -0.1423, -0.1220, -0.1339,  0.1224,
           -0.0781,  0.0473,  0.4268, -0.0551, -0.1086,  0.3161,  0.5081,
           -0.1031, -0.0036],
          [-0.0716, -0.1248, -0.0315, -0.1423, -0.1220, -0.1339,  0.1224,
           -0.0781,  0.0473,  0.4268, -0.0551, -0.1086,  0.3161,  0.5081,
           -0.1031, -0.0036],
          [-0.0716, -0.1248, -0.0315, -0.1423, -0.1220, -0.1339,  0.1224,
           -0.0781,  0.0473,  0.4268, -0.0551, -0.1086,  0.3161,  0.5081,
           -0.1031, -0.0036],
          [-0.0716, -0.1248, -0.0315, -0.1423, -0.1220, -0.1339,  0.1224,
           -0.0781,  0.0473,  0.4268, -0.0551, -0.1086,  0.3161,  0.5081,
           -0.1031, -0.0036],
          [-0.0716, -0.1248, -0.0315, -0.1423, -0.1220, -0.1339,  0.1224,
           

**MLP - down projection**  - Finally we'll take `x_mlp` and project `4C` back down to `C` using the weights in `mlp_proj`. Recall that we apply $x\_mlp = x\_mlp\_2 \cdot mlp\_proj^\top$.  Even though the layer weights are `4x16` the transpose in $XW^\top + B$ allows to project back down. We'll maintain the row repetion though this process for all the same reasons we've seen this far.

In [43]:
x_mlp = mlp_proj(x_mlp)
x_mlp.shape, x_mlp

(torch.Size([2, 8, 4]),
 tensor([[[ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559]],
 
         [[ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559],
          [ 0.2121, -0.1647, -0.0989,  0.1559]]], grad_fn=<ViewBackward0>))

#### Transformer - Residual (skip) connection 2

Once again our transformer uses a skip connection to allow for passing gradients through the Feed Forward, aka MLP, layer. Just like the first skip connection, functionally this is represented as

$y = f(x) + x$

To achieve this we simply sum the MLP input matrix `x` with the MLP output `x_mlp`.  As a reminder we'll print out `x`.  As you can see, because `x` was based on the tokens, it has a different value per row, so even though `x_mlp` has the same value per row, we'll result in a diverse set of weights.  With this you can see the power of skip connections passing through weights.

In [44]:
x

tensor([[[0.3712, 0.3928, 0.4148, 0.4371],
         [0.1812, 0.2028, 0.2248, 0.2471],
         [0.3612, 0.3828, 0.4048, 0.4271],
         [0.1412, 0.1628, 0.1848, 0.2071],
         [0.1112, 0.1328, 0.1548, 0.1771],
         [0.2712, 0.2928, 0.3148, 0.3371],
         [0.3812, 0.4028, 0.4248, 0.4471],
         [0.2412, 0.2628, 0.2848, 0.3071]],

        [[0.1312, 0.1528, 0.1748, 0.1971],
         [0.1212, 0.1428, 0.1648, 0.1871],
         [0.1012, 0.1228, 0.1448, 0.1671],
         [0.2512, 0.2728, 0.2948, 0.3171],
         [0.1112, 0.1328, 0.1548, 0.1771],
         [0.0712, 0.0928, 0.1148, 0.1371],
         [0.2112, 0.2328, 0.2548, 0.2771],
         [0.3012, 0.3228, 0.3448, 0.3671]]], grad_fn=<AddBackward0>)

In [45]:
x = x_mlp + x

x.shape, x

(torch.Size([2, 8, 4]),
 tensor([[[ 0.5833,  0.2281,  0.3159,  0.5930],
          [ 0.3933,  0.0381,  0.1259,  0.4030],
          [ 0.5733,  0.2181,  0.3059,  0.5830],
          [ 0.3533, -0.0019,  0.0859,  0.3630],
          [ 0.3233, -0.0319,  0.0559,  0.3330],
          [ 0.4833,  0.1281,  0.2159,  0.4930],
          [ 0.5933,  0.2381,  0.3259,  0.6030],
          [ 0.4533,  0.0981,  0.1859,  0.4630]],
 
         [[ 0.3433, -0.0119,  0.0759,  0.3530],
          [ 0.3333, -0.0219,  0.0659,  0.3430],
          [ 0.3133, -0.0419,  0.0459,  0.3230],
          [ 0.4633,  0.1081,  0.1959,  0.4730],
          [ 0.3233, -0.0319,  0.0559,  0.3330],
          [ 0.2833, -0.0719,  0.0159,  0.2930],
          [ 0.4233,  0.0681,  0.1559,  0.4330],
          [ 0.5133,  0.1581,  0.2459,  0.5230]]], grad_fn=<AddBackward0>))

#### Transformer - Final Layer Normalization 

The final step in the transformer is to aggregate and normalize before calculating the final projections. This layer is similar to the previous normalization layers. This layer will run the same normalization formula as before, but is its own independent layer as it has different inputs. Recall the formula is: 

$y = \frac{x - \mathbb{E}[x]}{\sqrt{\operatorname{Var}[x] + \epsilon}}$

Since this is the final layer, we will not have a residual connection so we do not need to branch `x`.  As with previous normalization layers, we'll once again see the rows unify in value as the distribution per row is the same. 

In [46]:
ln_f = nn.LayerNorm(n_embd)
ln_f

LayerNorm((4,), eps=1e-05, elementwise_affine=True)

In [47]:
x = ln_f(x)
x.shape, x

(torch.Size([2, 8, 4]),
 tensor([[[ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108]],
 
         [[ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108],
          [ 0.9509, -1.2532, -0.7086,  1.0108]]],
        grad_fn=<NativeLayerNormBackward0>))

### Output Layers AKA Model Head.

The combination of masked multi-head attention and feed forward, along with the normalization and residual connections is considered the "transformer".  In practice this layer is horizontally scaled to run many layers in parallel.  Once those layers are complete during the forward pass we then start the output process that results in `logits` which is a representation of the probability of each token being the next token given the input.  

This layer is also known as the model **head**, not to be confused with attention heads. This layer is called this because it is a small, task-specific module attached to a model’s shared backbone that maps hidden features to the final outputs.  In our example case, this is a linear layer mapping the backbone to vocab logits. The benefit of this structure is that you can use the shared hidden features and train different heads for different tasks without starting from scratch. An example would be a classifier head, or policy head in RL.

<img src="explainer_screenshots/output_layer.png" width="200">

For our head we want to map to a predicted token which we'll look at as `logits`. In the process to generate `logits` we take the normalized output `x` of the transformers, then project, using a linear layer, to the vocabulary resulting in a `B, T, vocab_size` matrix known as `logits`.  

In training, the `logits` are then compared with `y` to see how close the  model is to predicting the correct next token. For inference, the `logits` are then used to drive sampling which is how the next token is then derived. 


Instead of initializing weights this time around, we'll do **Weight Tying**.  Weight tying sets the output softmax matrix equal to the transpose of the input embedding matrix $W_{\text{out}} = W_e^\top$, forcing the model to “read” and “predict” in the same token space. This reduces parameters and acts as a useful prior, improving sample-efficiency and often perplexity by aligning input–output geometry. Modern LLMs have seemed to ditch this though to gain the extra capacity, but, for our example, we'll maintain it.





In [48]:
lm_head = nn.Linear(n_embd, vocab_size, bias=False)
# weight sharing scheme
lm_head.weight = wte.weight

lm_head, lm_head.weight


(Linear(in_features=4, out_features=36, bias=False),
 Parameter containing:
 tensor([[0.0100, 0.0200, 0.0300, 0.0400],
         [0.0200, 0.0300, 0.0400, 0.0500],
         [0.0300, 0.0400, 0.0500, 0.0600],
         [0.0400, 0.0500, 0.0600, 0.0700],
         [0.0500, 0.0600, 0.0700, 0.0800],
         [0.0600, 0.0700, 0.0800, 0.0900],
         [0.0700, 0.0800, 0.0900, 0.1000],
         [0.0800, 0.0900, 0.1000, 0.1100],
         [0.0900, 0.1000, 0.1100, 0.1200],
         [0.1000, 0.1100, 0.1200, 0.1300],
         [0.1100, 0.1200, 0.1300, 0.1400],
         [0.1200, 0.1300, 0.1400, 0.1500],
         [0.1300, 0.1400, 0.1500, 0.1600],
         [0.1400, 0.1500, 0.1600, 0.1700],
         [0.1500, 0.1600, 0.1700, 0.1800],
         [0.1600, 0.1700, 0.1800, 0.1900],
         [0.1700, 0.1800, 0.1900, 0.2000],
         [0.1800, 0.1900, 0.2000, 0.2100],
         [0.1900, 0.2000, 0.2100, 0.2200],
         [0.2000, 0.2100, 0.2200, 0.2300],
         [0.2100, 0.2200, 0.2300, 0.2400],
         [0.2200, 0.2

now let's check that the values are the same and that the underlying objects `data_ptr()` are

In [49]:
lm_head.weight is wte.weight, lm_head.weight.data_ptr() == wte.weight.data_ptr()

(True, True)

#### Output layer - LM Head aka logits
We now project `x` onto the vocabulary resulting in a `B X T X vocab_size` final array `logits`.  This output correlates with the 
probabilty of each output token given the input context.  The best way to read  this is:

(dimension 0) we have 2 batches B, 
(dimension 1) each batch has an example for each value between 1 and context length T 
(dimension 2) for each example we see the probability of each token in our vocabulary

Since our `x` at this point has the same values across the row, for every row, we fully expect that our logits will have the same value.  In practice this means that our model will have the same probability of a token output as the 'next token' regardless of the preceeding text, meaning it's shit. Luckily backpropogation has a way of updating this so that with enough data and time the probabilities change. 

In [50]:
x

tensor([[[ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108]],

        [[ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108],
         [ 0.9509, -1.2532, -0.7086,  1.0108]]],
       grad_fn=<NativeLayerNormBackward0>)

In [51]:
logits = lm_head(x)

logits.shape, logits

(torch.Size([2, 8, 36]),
 tensor([[[0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036],
          [0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036],
          [0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
           0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0

## Loss calculation
Now we have to see how good our ~shit~ prediction is.  Since we haven't done training, and we saw that regardless of example we had the same exact logit values, we can expect it's bad. That said, we need to know how bad. For this example we'll use cross entropy, also known as the negative log likelihood of the softmax.  Our loss calculates

$$
\ell_i=-\log\big(\mathrm{softmax}(z_i)\_{y_i}\big)
= -z_{i,y_i}+\log\!\sum_{c=1}^C e^{z_{i,c}},
$$


To calculate loss we'll pass in the calculated `logits` and our next tokens stored in `y`. The cross entropy function does not respect batches so we'll flatten the `B` dimension for both `logits` and `y`

In [52]:
y_flat = y.view(-1)
y_flat.shape, y_flat

(torch.Size([16]),
 tensor([15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0]))

In [53]:
logits_flat = logits.view(-1, logits.size(-1))
logits_flat.shape, logits_flat

(torch.Size([16, 36]),
 tensor([[0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036],
         [0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036],
         [0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,
          0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036, 0.0036,

In [54]:
loss = F.cross_entropy(logits_flat, y_flat)
loss.shape, loss

(torch.Size([]), tensor(3.5835, grad_fn=<NllLossBackward0>))

## Back Propogation

We now know just how well the current model, with its weights and biases, does predicting the next token given the input context. We now need to know how to change the different weigths and biases to improve the formula.  We could do this by guessing through making minor changes and seeing what improves, or we can think through this more critically.

If you review the chain of layers above, you can see that it's a series of formulas.  We can think of this as $f(g(x))$, except with many many more layers and complexities.  Since this is a formula, we can dig into our math toolbox and find a better way to determine what parts need to update.  Recall that in our calculus we learned that differentiation tells us the rate of change in a graph.  So if we treat the loss function $\mathcal{L}$ as $\mathcal{L}(f(g(x)))$ taking the partial differential 

$\delta=\partial \mathcal{L}/\partial h$

at each layer will give us the impact of each weight/bias on our final out (alebit the inverse since our loss function is the negative log likelihood). 

Lucky for us, each layer of our model already has a placeholder for the partial differential called the **Gradient**. We'll use this field to store it.  We'll start by first zeroing out the gradients. We do this because of the nature of handling partial differentials for multiple dependencies. Recall that in mulitple places we had a formula structure of 

$a+b=c ; a+c= d$

In this case $a$ has 2 dependencies and determining the partial derivative of $\partial d / \partial a$ requires understanding both the path from $d$ and $c$.  To determine the true impact of a we would sum both partial derivatives together.  Because of this property, the tool we use, the built in `.backwards()` automatically sums gradients, `+=`, so if we do not set the gradient to `0` we then end up with eroneous gradients. 

Finally, we start `.backwards` from the `loss`, not `logits` as our goal is to minimize loss, we need to ensure we are looking at the calcualations that impact loss which requires the whole forward pass to be able to generated the prediction `logits_flat`.  If we think of it as $\mathcal{L}(f(x))$ where $f(x)$ is the forward pass to generate logits, then a simple chain rule is applied:

${\partial}/{\partial x} =  \mathcal{L}'(f(x)) f'(x)$

Lets start by zeroing the gradients and leaning on pytorch to calculate the gradients for us. We'll also validate the gradients were `none`.

In [55]:
lm_head.zero_grad()
ln_f.zero_grad()
mlp_proj.zero_grad()
mlp_fc.zero_grad()
ln_2.zero_grad()
c_proj.zero_grad()
c_attn.zero_grad()
ln_1.zero_grad()
wpe.zero_grad()
wte.zero_grad()

# validate gradients
lm_head.weight.grad, ln_f.weight.grad

(None, None)

In [56]:
loss.backward()

**Auto-Diff** - Now let's see the magic of the gradients populate.  This magic is called auto-differentiation, or auto-diff for short. This allows us to not have to write write many layers of nasty code to do the differentiation for us, but, if you're a sadist, you can surely find people who have written out that code (it's not too bad since you just do one layer at a time). 

In [57]:
lm_head.weight.grad, ln_f.weight.grad

(tensor([[-0.0924,  0.1218,  0.0689, -0.0983],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
         [-0.0330,  0.0435,  0.0246, -0.0351],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0330,  0.0435,  0.0246, -0.0351],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0330,  0.0435,  0.0246, -0.0351],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
         [-0.

## Learning
The process of learning now requires us to update our weights based on this gradient. To really feel the "back propogation" we'll start with the last layer and work backwards, though, since we have all of the gradients calculated already, the order does not matter. Recall that our loss function is the negative log likelihood ratio so our gradient signs are flipped.  If a parameter is important, the gradient will be more negaitve, and vice versa. The gradients are a ratio of importance of each parameter and we need to know how much of that gradient to apply to our weights. This "how much" is referred to as the *learning rate*.  In modern training learning rate schedulers and optimzers are used to vary the rate and application by layer and by training round.  We'll keep it simple and use an astronomically high learning rate of `1.000` which applies the gradient directly to the weights via a `-=`. Gradient for the weights and the biases is different as the partial differential with respect to each is different. We need to remember in the layers with bias to apply it to both.

*Note that since our vocab is very small, our context is small, and our batch is small, relatively our model is very deep so we will see a lot of exceptionally small gradients*

In [58]:
## Huge learning rate to emphasize
learning_rate = 1.000 

### Output Layer
Let's start with our output layer.  If you recall we did *weight tying* of our head `lm_head` to our token projection `wte`.  Because of this applying loss to the `lm_head` will automatically apply it to `wte`. Since we're doing this manually and not through an optimizer we need to be careful not to apply it twice. Since we're starting with the last layer first, the head, we'll see that the gradients are equal and that updating just the `lm_head` weights updates `wte` weights also.

In [59]:
lm_head.weight.grad is wte.weight.grad, lm_head.weight.grad, wte.weight.grad

(True,
 tensor([[-0.0924,  0.1218,  0.0689, -0.0983],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
         [-0.0330,  0.0435,  0.0246, -0.0351],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0330,  0.0435,  0.0246, -0.0351],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0330,  0.0435,  0.0246, -0.0351],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [ 0.0264, -0.0348, -0.0197,  0.0281],
         [-0.0924,  0.1218,  0.0689, -0.0983],
      

In [60]:
with torch.no_grad():
    lm_head.weight -= learning_rate * lm_head.weight.grad
lm_head.weight is wte.weight, lm_head.weight, wte.weight

(True,
 Parameter containing:
 tensor([[ 0.1024, -0.1018, -0.0389,  0.1383],
         [-0.0064,  0.0648,  0.0597,  0.0219],
         [ 0.0036,  0.0748,  0.0697,  0.0319],
         [ 0.0136,  0.0848,  0.0797,  0.0419],
         [ 0.0236,  0.0948,  0.0897,  0.0519],
         [ 0.1524, -0.0518,  0.0111,  0.1883],
         [ 0.1030,  0.0365,  0.0654,  0.1351],
         [ 0.0536,  0.1248,  0.1197,  0.0819],
         [ 0.0636,  0.1348,  0.1297,  0.0919],
         [ 0.1924, -0.0118,  0.0511,  0.2283],
         [ 0.0836,  0.1548,  0.1497,  0.1119],
         [ 0.1530,  0.0865,  0.1154,  0.1851],
         [ 0.1036,  0.1748,  0.1697,  0.1319],
         [ 0.1730,  0.1065,  0.1354,  0.2051],
         [ 0.1236,  0.1948,  0.1897,  0.1519],
         [ 0.2524,  0.0482,  0.1111,  0.2883],
         [ 0.1436,  0.2148,  0.2097,  0.1719],
         [ 0.1536,  0.2248,  0.2197,  0.1819],
         [ 0.1636,  0.2348,  0.2297,  0.1919],
         [ 0.1736,  0.2448,  0.2397,  0.2019],
         [ 0.3024,  0.0982,  0

### Transformer - Final Layer Normalization
Now let's move to the layer normalization.  In our forward pass we did not change the weights, but in training we still will as the normalization can impact the outputs.  Additionally layer normalization has bias so we have to remember to add it. 

In [61]:
ln_f.weight, ln_f.bias

(Parameter containing:
 tensor([1., 1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0.], requires_grad=True))

In [62]:
ln_f.weight.grad, ln_f.bias.grad

(tensor([ 0.0410, -0.0540, -0.0306,  0.0436]),
 tensor([0.0431, 0.0431, 0.0431, 0.0431]))

In [63]:
with torch.no_grad():
    ln_f.weight -= learning_rate * ln_f.weight.grad
    ln_f.bias   -= learning_rate * ln_f.bias.grad
ln_f.weight, ln_f.bias

(Parameter containing:
 tensor([0.9590, 1.0540, 1.0306, 0.9564], requires_grad=True),
 Parameter containing:
 tensor([-0.0431, -0.0431, -0.0431, -0.0431], requires_grad=True))

### Tranformer - Feed Froward Updates
In feed forward we had 3 laysers: the upward projection `mlp_fc`, the tanh layer `mlp_gelu`, and the downward projection `mlp_proj`.  The tanh layer normalizes each element using tanh so it has no trainable parameters, or weights, so there is no updates to be made, the gradient passes through it. But the other two layers have both weights and biases that we can update.  

One thing you may notice is that the gradients are exceptionally small in some cases, leading to minimal updates

In [64]:
mlp_proj.weight.grad, mlp_proj.bias.grad, mlp_fc.weight.grad,mlp_fc.bias.grad

(tensor([[ 5.9999e-10,  1.0460e-09,  2.6364e-10,  1.1927e-09,  1.0229e-09,
           1.1223e-09, -1.0259e-09,  6.5427e-10, -3.9661e-10, -3.5777e-09,
           4.6170e-10,  9.1048e-10, -2.6494e-09, -4.2588e-09,  8.6449e-10,
           3.0261e-11],
         [-1.8000e-09, -3.1380e-09, -7.9090e-10, -3.5782e-09, -3.0686e-09,
          -3.3668e-09,  3.0778e-09, -1.9628e-09,  1.1898e-09,  1.0733e-08,
          -1.3851e-09, -2.7314e-09,  7.9483e-09,  1.2776e-08, -2.5935e-09,
          -9.0780e-11],
         [-1.7333e-09, -3.0218e-09, -7.6161e-10, -3.4456e-09, -2.9550e-09,
          -3.2421e-09,  2.9638e-09, -1.8901e-09,  1.1458e-09,  1.0336e-08,
          -1.3338e-09, -2.6303e-09,  7.6539e-09,  1.2303e-08, -2.4974e-09,
          -8.7420e-11],
         [ 2.6666e-09,  4.6489e-09,  1.1717e-09,  5.3010e-09,  4.5461e-09,
           4.9878e-09, -4.5597e-09,  2.9079e-09, -1.7627e-09, -1.5901e-08,
           2.0520e-09,  4.0466e-09, -1.1775e-08, -1.8928e-08,  3.8422e-09,
           1.3450e-10]]),
 t

In [65]:
with torch.no_grad():
    mlp_proj.weight -= learning_rate * mlp_proj.weight.grad
    mlp_proj.bias -= learning_rate * mlp_proj.bias.grad
    mlp_fc.weight -= learning_rate * mlp_fc.weight.grad
    mlp_fc.bias -= learning_rate * mlp_fc.bias.grad
mlp_proj.weight, mlp_proj.bias, mlp_fc.weight,mlp_fc.bias

(Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040, 0.0050, 0.0060, 0.0070, 0.0080, 0.0090,
          0.0100, 0.0110, 0.0120, 0.0130, 0.0140, 0.0150, 0.0160],
         [0.0020, 0.0050, 0.0070, 0.0090, 0.0110, 0.0130, 0.0150, 0.0170, 0.0190,
          0.0210, 0.0230, 0.0250, 0.0270, 0.0290, 0.0310, 0.0330],
         [0.0030, 0.0090, 0.0120, 0.0150, 0.0180, 0.0210, 0.0240, 0.0270, 0.0300,
          0.0330, 0.0360, 0.0390, 0.0420, 0.0450, 0.0480, 0.0510],
         [0.0040, 0.0140, 0.0180, 0.0220, 0.0260, 0.0300, 0.0340, 0.0380, 0.0420,
          0.0460, 0.0500, 0.0540, 0.0580, 0.0620, 0.0660, 0.0700]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.2019, -0.1857, -0.1312,  0.1118], requires_grad=True),
 Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040],
         [0.0020, 0.0050, 0.0070, 0.0090],
         [0.0030, 0.0090, 0.0120, 0.0150],
         [0.0040, 0.0140, 0.0180, 0.0220],
         [0.0050, 0.0200, 0.0250, 0.0300],
         [0.0060, 0.027

### Tranformer - Layer Norm 2
Now on to the next layer normalization.  Similarly we have both weights and biases to update again.  We can see once again these are exceptionally small gradients. 

In [66]:
ln_2.weight.grad, ln_2.bias.grad

(tensor([ 4.9795e-11,  1.2989e-10, -1.4203e-10, -4.8610e-10]),
 tensor([-3.7632e-11, -2.8803e-10, -3.2567e-10, -3.6330e-10]))

In [67]:
with torch.no_grad():
    ln_2.weight -= learning_rate * ln_2.weight.grad
    ln_2.bias   -= learning_rate * ln_2.bias.grad
ln_2.weight, ln_2.bias

(Parameter containing:
 tensor([1., 1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([3.7632e-11, 2.8803e-10, 3.2567e-10, 3.6330e-10], requires_grad=True))

### Tranformer - Flash Attention
Even though flash attention was excpetionally complicated with multiple attnetion heads and the query, key, and value, we only have 2 sets of weights, and biases, to update: `c_attn`, `c_proj`.  Back propogation aggregated the different impacts across on each.  

Similar to what we've seen this far, these gradiens are very small but we can see large order of magnitudedifferences within the values starting to suggest how they impact the ouputs. 

In [68]:
c_attn.weight.grad,c_attn.bias.grad, c_proj.weight.grad, c_proj.bias.grad,

(tensor([[ 2.0336e-20,  6.7788e-21, -6.7789e-21, -2.0336e-20],
         [ 2.6692e-20,  8.8972e-21, -8.8973e-21, -2.6692e-20],
         [-1.5235e-19, -5.0782e-20,  5.0782e-20,  1.5235e-19],
         [-1.8786e-19, -6.2619e-20,  6.2619e-20,  1.8786e-19],
         [ 2.5421e-21,  8.4737e-22, -8.4735e-22, -2.5421e-21],
         [ 5.8467e-21,  1.9490e-21, -1.9489e-21, -5.8468e-21],
         [-4.4673e-20, -1.4891e-20,  1.4891e-20,  4.4673e-20],
         [-6.6437e-20, -2.2145e-20,  2.2146e-20,  6.6437e-20],
         [ 4.6316e-11,  1.5439e-11, -1.5439e-11, -4.6316e-11],
         [ 2.5493e-10,  8.4978e-11, -8.4978e-11, -2.5493e-10],
         [ 3.0125e-10,  1.0042e-10, -1.0042e-10, -3.0125e-10],
         [ 3.4757e-10,  1.1586e-10, -1.1586e-10, -3.4757e-10]]),
 tensor([-1.5309e-20, -2.0093e-20,  1.1468e-19,  1.4141e-19, -1.9136e-21,
         -4.4013e-21,  3.3629e-20,  5.0012e-20, -3.4865e-11, -1.9191e-10,
         -2.2677e-10, -2.6164e-10]),
 tensor([[-4.9517e-10, -5.8770e-10, -6.8774e-10, -7.9527e

In [69]:
with torch.no_grad():
    c_attn.weight -= learning_rate * c_attn.weight.grad
    c_attn.bias -= learning_rate * c_attn.bias.grad
    c_proj.weight -= learning_rate * c_proj.weight.grad
    c_proj.bias -= learning_rate * c_proj.bias.grad
c_attn.weight,c_attn.bias, c_proj.weight, c_proj.bias,

(Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040],
         [0.0020, 0.0050, 0.0070, 0.0090],
         [0.0030, 0.0090, 0.0120, 0.0150],
         [0.0040, 0.0140, 0.0180, 0.0220],
         [0.0050, 0.0200, 0.0250, 0.0300],
         [0.0060, 0.0270, 0.0330, 0.0390],
         [0.0070, 0.0350, 0.0420, 0.0490],
         [0.0080, 0.0440, 0.0520, 0.0600],
         [0.0090, 0.0540, 0.0630, 0.0720],
         [0.0100, 0.0650, 0.0750, 0.0850],
         [0.0110, 0.0770, 0.0880, 0.0990],
         [0.0120, 0.0900, 0.1020, 0.1140]], requires_grad=True),
 Parameter containing:
 tensor([ 1.5309e-20,  2.0093e-20, -1.1468e-19, -1.4141e-19,  1.9136e-21,
          4.4013e-21, -3.3629e-20, -5.0012e-20,  3.4865e-11,  1.9191e-10,
          2.2677e-10,  2.6164e-10], requires_grad=True),
 Parameter containing:
 tensor([[0.0010, 0.0020, 0.0030, 0.0040],
         [0.0020, 0.0050, 0.0070, 0.0090],
         [0.0030, 0.0090, 0.0120, 0.0150],
         [0.0040, 0.0140, 0.0180, 0.0220]], requires_grad=T

### Tranformer - Layer Norm 1
Now on to the first layer normalization in a transfomer.  Similarly to the other layer normalizations we have both weights and biases to update again.  We can see once again these are exceptionally small gradients. 

In [70]:
ln_1.weight.grad, ln_1.bias.grad

(tensor([ 1.0451e-11,  2.4516e-11, -2.8000e-11, -9.4450e-11]),
 tensor([-7.8670e-12, -5.5366e-11, -6.3233e-11, -7.1100e-11]))

In [71]:
with torch.no_grad():
    ln_1.weight -= learning_rate * ln_1.weight.grad
    ln_1.bias   -= learning_rate * ln_1.bias.grad
ln_1.weight, ln_1.bias

(Parameter containing:
 tensor([1., 1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([7.8670e-12, 5.5366e-11, 6.3233e-11, 7.1100e-11], requires_grad=True))

### Input Layer 
Now on to the input layer.  Recall that this layer had 2 components: the positional embeddings `wpe` and the token embeddings `wte`. We used *weight tying* to tie our token project `wte` with our head `lm_head` which has already been updated so we do not need to touch it.  We do however still need to update our positional embeddings.   Remember that `wpe` as an embedding has only weights and no biases. We'll see that once again we have relatively small gradients but we can see that they do very a few orders of magnitude showing that even with our small vocab and context, the position are starting to show some impact. 

*Note that this is the final layer of update we need to make.  Tokenization and data loading is separate from the model and does not impact loss*

In [72]:
wpe.weight.grad

tensor([[-5.4412e-09,  2.6099e-09,  1.2057e-08, -9.2258e-09],
        [ 4.7085e-10,  7.8105e-09,  8.5616e-09,  1.7835e-09],
        [ 3.3993e-09, -1.0549e-08, -6.4291e-09,  5.3988e-10],
        [ 1.7873e-09, -9.6942e-09, -4.3349e-09,  1.0659e-09],
        [ 6.7273e-11,  2.6944e-08,  6.8373e-09, -4.0460e-09],
        [ 6.3407e-09, -8.8710e-09, -8.3426e-09, -3.0296e-10],
        [ 8.0353e-09, -4.4592e-09,  6.3201e-09, -6.1710e-09],
        [-1.9755e-08,  1.7230e-08,  8.4270e-09, -1.8940e-08]])

In [73]:
with torch.no_grad():
    wpe.weight -= learning_rate * wpe.weight.grad
wpe.weight

Parameter containing:
tensor([[0.0100, 0.0200, 0.0300, 0.0400],
        [0.0200, 0.0300, 0.0400, 0.0500],
        [0.0300, 0.0400, 0.0500, 0.0600],
        [0.0400, 0.0500, 0.0600, 0.0700],
        [0.0500, 0.0600, 0.0700, 0.0800],
        [0.0600, 0.0700, 0.0800, 0.0900],
        [0.0700, 0.0800, 0.0900, 0.1000],
        [0.0800, 0.0900, 0.1000, 0.1100]], requires_grad=True)

## Forward Pass with Updated Weights

Now that we have the updated weights for each layer, let's do another forward pass and compare the loss. Since each layer was previously explained we will instead focus on just showing the outputs of the different layers and the final loss. If you want, you can check the previous outputs in the cached cell outputs above and compare them to see how the weight changes impacted the values at each layer. 

One key sign that our weights were updated is that you'll see quickly that the values at each layer are no longer repeated.  

### Data Re-loading
Repulling to a new `x_2`. We'll keep `y` to emphasize the same examples are being used. 

In [74]:
x_2 = tok_for_training[:-1].view(B, T)
x_2, y

(tensor([[35, 15, 32,  9,  5, 20, 30, 15],
         [11,  9,  6, 20,  5,  0, 13, 21]]),
 tensor([[15, 32,  9,  5, 20, 30, 15, 11],
         [ 9,  6, 20,  5,  0, 13, 21,  0]]))

### Input Layer
Note that in `wte` you can already see the impact of updated weights with the negative values. This was not originally present. 

In [75]:
pos = torch.arange(0, T, dtype=torch.long)
pos_emb = wpe(pos)
print(pos_emb.shape, pos_emb)

torch.Size([8, 4]) tensor([[0.0100, 0.0200, 0.0300, 0.0400],
        [0.0200, 0.0300, 0.0400, 0.0500],
        [0.0300, 0.0400, 0.0500, 0.0600],
        [0.0400, 0.0500, 0.0600, 0.0700],
        [0.0500, 0.0600, 0.0700, 0.0800],
        [0.0600, 0.0700, 0.0800, 0.0900],
        [0.0700, 0.0800, 0.0900, 0.1000],
        [0.0800, 0.0900, 0.1000, 0.1100]], grad_fn=<EmbeddingBackward0>)


In [76]:
tok_emb = wte(x_2)
print(tok_emb.shape, tok_emb)

torch.Size([2, 8, 4]) tensor([[[ 0.3336,  0.4048,  0.3997,  0.3619],
         [ 0.2524,  0.0482,  0.1111,  0.2883],
         [ 0.3630,  0.2965,  0.3254,  0.3951],
         [ 0.1924, -0.0118,  0.0511,  0.2283],
         [ 0.1524, -0.0518,  0.0111,  0.1883],
         [ 0.3024,  0.0982,  0.1611,  0.3383],
         [ 0.3430,  0.2765,  0.3054,  0.3751],
         [ 0.2524,  0.0482,  0.1111,  0.2883]],

        [[ 0.1530,  0.0865,  0.1154,  0.1851],
         [ 0.1924, -0.0118,  0.0511,  0.2283],
         [ 0.1030,  0.0365,  0.0654,  0.1351],
         [ 0.3024,  0.0982,  0.1611,  0.3383],
         [ 0.1524, -0.0518,  0.0111,  0.1883],
         [ 0.1024, -0.1018, -0.0389,  0.1383],
         [ 0.1730,  0.1065,  0.1354,  0.2051],
         [ 0.2530,  0.1865,  0.2154,  0.2851]]], grad_fn=<EmbeddingBackward0>)


In [77]:
x = tok_emb + pos_emb
print(x)

tensor([[[ 0.3436,  0.4248,  0.4297,  0.4019],
         [ 0.2724,  0.0782,  0.1511,  0.3383],
         [ 0.3930,  0.3365,  0.3754,  0.4551],
         [ 0.2324,  0.0382,  0.1111,  0.2983],
         [ 0.2024,  0.0082,  0.0811,  0.2683],
         [ 0.3624,  0.1682,  0.2411,  0.4283],
         [ 0.4130,  0.3565,  0.3954,  0.4751],
         [ 0.3324,  0.1382,  0.2111,  0.3983]],

        [[ 0.1630,  0.1065,  0.1454,  0.2251],
         [ 0.2124,  0.0182,  0.0911,  0.2783],
         [ 0.1330,  0.0765,  0.1154,  0.1951],
         [ 0.3424,  0.1482,  0.2211,  0.4083],
         [ 0.2024,  0.0082,  0.0811,  0.2683],
         [ 0.1624, -0.0318,  0.0411,  0.2283],
         [ 0.2430,  0.1865,  0.2254,  0.3051],
         [ 0.3330,  0.2765,  0.3154,  0.3951]]], grad_fn=<AddBackward0>)


### Transformer - Layer Normalization
If you recall, the values here were all uniform. The updated weights and changed input now result in changes, showing the impact of our learning.

In [78]:
x_norm = ln_1(x)
print(x_norm.shape, x_norm)

torch.Size([2, 8, 4]) tensor([[[-1.6418,  0.7220,  0.8638,  0.0559],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.0703, -1.2472, -0.3404,  1.5172],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.0703, -1.2472, -0.3404,  1.5172],
         [ 0.6150, -1.2983, -0.5800,  1.2633]],

        [[ 0.0703, -1.2472, -0.3404,  1.5172],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.0703, -1.2472, -0.3404,  1.5172],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.6150, -1.2983, -0.5800,  1.2633],
         [ 0.0703, -1.2472, -0.3404,  1.5172],
         [ 0.0703, -1.2472, -0.3404,  1.5172]]],
       grad_fn=<NativeLayerNormBackward0>)


### Transformer - Flash Attention

In [79]:
B, T, C = x_norm.size()
qkv = c_attn(x_norm)
print(qkv.shape, qkv)

torch.Size([2, 8, 12]) tensor([[[ 0.0026,  0.0069,  0.0128,  0.0203,  0.0295,  0.0403,  0.0528,
           0.0669,  0.0827,  0.1001,  0.1191,  0.1398],
         [ 0.0013,  0.0020,  0.0021,  0.0016,  0.0005, -0.0012, -0.0036,
          -0.0066, -0.0102, -0.0144, -0.0192, -0.0246],
         [ 0.0026,  0.0052,  0.0077,  0.0101,  0.0124,  0.0147,  0.0169,
           0.0190,  0.0211,  0.0231,  0.0250,  0.0268],
         [ 0.0013,  0.0020,  0.0021,  0.0016,  0.0005, -0.0012, -0.0036,
          -0.0066, -0.0102, -0.0144, -0.0192, -0.0246],
         [ 0.0013,  0.0020,  0.0021,  0.0016,  0.0005, -0.0012, -0.0036,
          -0.0066, -0.0102, -0.0144, -0.0192, -0.0246],
         [ 0.0013,  0.0020,  0.0021,  0.0016,  0.0005, -0.0012, -0.0036,
          -0.0066, -0.0102, -0.0144, -0.0192, -0.0246],
         [ 0.0026,  0.0052,  0.0077,  0.0101,  0.0124,  0.0147,  0.0169,
           0.0190,  0.0211,  0.0231,  0.0250,  0.0268],
         [ 0.0013,  0.0020,  0.0021,  0.0016,  0.0005, -0.0012, -0.0036,
 

In [80]:
q,k,v = qkv.split(n_embd, dim=2)
q = q.view(B, T, n_head, C // n_head).transpose(1, 2)
k = k.view(B, T, n_head, C // n_head).transpose(1, 2)
v = v.view(B, T, n_head, C // n_head).transpose(1, 2)
fa = F.scaled_dot_product_attention(q, k, v, is_causal=True)
fa = fa.transpose(1, 2).contiguous().view(B, T, C)
x_norm = c_proj(fa)
print(x_norm.shape, x_norm)

torch.Size([2, 8, 4]) tensor([[[ 1.1991e-03,  2.7571e-03,  4.6740e-03,  6.9499e-03],
         [ 5.0219e-04,  1.1547e-03,  1.9577e-03,  2.9112e-03],
         [ 4.1812e-04,  9.6148e-04,  1.6301e-03,  2.4241e-03],
         [ 2.6479e-04,  6.0894e-04,  1.0325e-03,  1.5355e-03],
         [ 1.7287e-04,  3.9759e-04,  6.7422e-04,  1.0028e-03],
         [ 1.1159e-04,  2.5669e-04,  4.3536e-04,  6.4765e-04],
         [ 1.3139e-04,  3.0223e-04,  5.1258e-04,  7.6247e-04],
         [ 9.0535e-05,  2.0829e-04,  3.5334e-04,  5.2570e-04]],

        [[ 2.4960e-04,  5.7407e-04,  9.7348e-04,  1.4479e-03],
         [ 2.7383e-05,  6.3117e-05,  1.0726e-04,  1.5985e-04],
         [ 1.0147e-04,  2.3348e-04,  3.9608e-04,  5.8930e-04],
         [ 2.7383e-05,  6.3118e-05,  1.0726e-04,  1.5985e-04],
         [-1.7061e-05, -3.9076e-05, -6.5987e-05, -9.7759e-05],
         [-4.6690e-05, -1.0721e-04, -1.8149e-04, -2.6950e-04],
         [-4.3421e-06, -9.8311e-06, -1.6408e-05, -2.4038e-05],
         [ 2.7404e-05,  6.3165e

### Transformer - Residual connection

In [81]:
x = x + x_norm
print(x.shape, x)

torch.Size([2, 8, 4]) tensor([[[ 0.3448,  0.4276,  0.4344,  0.4089],
         [ 0.2730,  0.0793,  0.1531,  0.3412],
         [ 0.3934,  0.3374,  0.3770,  0.4575],
         [ 0.2327,  0.0388,  0.1121,  0.2998],
         [ 0.2026,  0.0086,  0.0818,  0.2693],
         [ 0.3626,  0.1684,  0.2415,  0.4289],
         [ 0.4131,  0.3568,  0.3959,  0.4759],
         [ 0.3325,  0.1384,  0.2115,  0.3988]],

        [[ 0.1633,  0.1071,  0.1464,  0.2265],
         [ 0.2125,  0.0182,  0.0912,  0.2784],
         [ 0.1331,  0.0767,  0.1158,  0.1957],
         [ 0.3425,  0.1482,  0.2212,  0.4084],
         [ 0.2024,  0.0081,  0.0810,  0.2682],
         [ 0.1624, -0.0319,  0.0409,  0.2280],
         [ 0.2430,  0.1865,  0.2254,  0.3051],
         [ 0.3330,  0.2766,  0.3155,  0.3953]]], grad_fn=<AddBackward0>)


### Transformer - Layer Normalization 2

In [82]:
x_norm_2 = ln_2(x)
print(x_norm_2.shape, x_norm_2)

torch.Size([2, 8, 4]) tensor([[[-1.6641,  0.6664,  0.8576,  0.1401],
         [ 0.6019, -1.2987, -0.5748,  1.2716],
         [ 0.0479, -1.2422, -0.3302,  1.5246],
         [ 0.6081, -1.2985, -0.5772,  1.2677],
         [ 0.6105, -1.2985, -0.5782,  1.2662],
         [ 0.6121, -1.2984, -0.5788,  1.2652],
         [ 0.0632, -1.2456, -0.3372,  1.5196],
         [ 0.6126, -1.2984, -0.5790,  1.2648]],

        [[ 0.0568, -1.2442, -0.3343,  1.5217],
         [ 0.6143, -1.2983, -0.5797,  1.2637],
         [ 0.0648, -1.2460, -0.3379,  1.5191],
         [ 0.6143, -1.2983, -0.5797,  1.2637],
         [ 0.6154, -1.2983, -0.5801,  1.2630],
         [ 0.6162, -1.2983, -0.5804,  1.2625],
         [ 0.0705, -1.2472, -0.3405,  1.5172],
         [ 0.0688, -1.2469, -0.3397,  1.5177]]],
       grad_fn=<NativeLayerNormBackward0>)


### Transformer - Feed Forward

In [83]:
x_mlp = mlp_fc(x_norm_2)
x_mlp = mlp_gelu(x_mlp)
x_mlp = mlp_proj(x_mlp)
print(x_mlp.shape, x_mlp)

torch.Size([2, 8, 4]) tensor([[[ 0.2124, -0.1642, -0.0980,  0.1571],
         [ 0.2018, -0.1862, -0.1324,  0.1096],
         [ 0.2048, -0.1800, -0.1227,  0.1229],
         [ 0.2018, -0.1863, -0.1325,  0.1095],
         [ 0.2018, -0.1863, -0.1325,  0.1094],
         [ 0.2017, -0.1863, -0.1325,  0.1094],
         [ 0.2047, -0.1802, -0.1230,  0.1226],
         [ 0.2017, -0.1863, -0.1325,  0.1094]],

        [[ 0.2047, -0.1801, -0.1229,  0.1227],
         [ 0.2017, -0.1863, -0.1326,  0.1093],
         [ 0.2047, -0.1802, -0.1230,  0.1225],
         [ 0.2017, -0.1863, -0.1326,  0.1093],
         [ 0.2017, -0.1863, -0.1326,  0.1093],
         [ 0.2017, -0.1863, -0.1326,  0.1093],
         [ 0.2046, -0.1803, -0.1231,  0.1224],
         [ 0.2046, -0.1803, -0.1231,  0.1224]]], grad_fn=<ViewBackward0>)


### Transformer - Residual (skip) connection 2

In [84]:
x = x_mlp + x
print(x.shape, x)

torch.Size([2, 8, 4]) tensor([[[ 0.5572,  0.2634,  0.3363,  0.5660],
         [ 0.4747, -0.1069,  0.0207,  0.4508],
         [ 0.5982,  0.1574,  0.2543,  0.5805],
         [ 0.4345, -0.1475, -0.0203,  0.4093],
         [ 0.4044, -0.1777, -0.0507,  0.3787],
         [ 0.5643, -0.0179,  0.1090,  0.5383],
         [ 0.6178,  0.1766,  0.2729,  0.5984],
         [ 0.5343, -0.0479,  0.0789,  0.5082]],

        [[ 0.3680, -0.0731,  0.0235,  0.3493],
         [ 0.4142, -0.1681, -0.0414,  0.3878],
         [ 0.3378, -0.1035, -0.0072,  0.3182],
         [ 0.5442, -0.0381,  0.0886,  0.5178],
         [ 0.4042, -0.1782, -0.0515,  0.3775],
         [ 0.3641, -0.2183, -0.0917,  0.3373],
         [ 0.4476,  0.0062,  0.1022,  0.4275],
         [ 0.5377,  0.0963,  0.1924,  0.5177]]], grad_fn=<AddBackward0>)


### Transformer - Final Layer Normalization

In [85]:
x = ln_f(x)
print(x.shape, x)

torch.Size([2, 8, 4]) tensor([[[ 0.8657, -1.3647, -0.7721,  0.9262],
         [ 0.9450, -1.3417, -0.8013,  0.8533],
         [ 0.9439, -1.3420, -0.8009,  0.8543],
         [ 0.9474, -1.3409, -0.8022,  0.8510],
         [ 0.9483, -1.3406, -0.8025,  0.8501],
         [ 0.9490, -1.3404, -0.8027,  0.8495],
         [ 0.9481, -1.3406, -0.8023,  0.8502],
         [ 0.9492, -1.3404, -0.8028,  0.8493]],

        [[ 0.9464, -1.3412, -0.8017,  0.8519],
         [ 0.9498, -1.3401, -0.8030,  0.8487],
         [ 0.9486, -1.3405, -0.8025,  0.8498],
         [ 0.9498, -1.3401, -0.8030,  0.8487],
         [ 0.9502, -1.3400, -0.8031,  0.8483],
         [ 0.9505, -1.3399, -0.8033,  0.8480],
         [ 0.9501, -1.3400, -0.8030,  0.8483],
         [ 0.9496, -1.3401, -0.8029,  0.8488]]],
       grad_fn=<NativeLayerNormBackward0>)


### Output Layers AKA Model Head

In [86]:
logits = lm_head(x)
print(logits.shape, logits)

torch.Size([2, 8, 36]) tensor([[[ 0.3858, -0.1198, -0.1232, -0.1267, -0.1301,  0.3685,  0.1140,
          -0.1405, -0.1439,  0.3547, -0.1508,  0.0968, -0.1577,  0.0899,
          -0.1646,  0.3340, -0.1715, -0.1750, -0.1784, -0.1819,  0.3168,
           0.0623, -0.1922, -0.1957, -0.1991, -0.2026, -0.2060, -0.2095,
          -0.2129, -0.2164,  0.0312, -0.2233,  0.0243, -0.2302, -0.2336,
          -0.2370],
         [ 0.3826, -0.1221, -0.1256, -0.1290, -0.1325,  0.3654,  0.1113,
          -0.1428, -0.1463,  0.3516, -0.1532,  0.0940, -0.1601,  0.0871,
          -0.1669,  0.3309, -0.1738, -0.1773, -0.1807, -0.1842,  0.3137,
           0.0596, -0.1945, -0.1980, -0.2014, -0.2049, -0.2083, -0.2118,
          -0.2152, -0.2186,  0.0286, -0.2255,  0.0217, -0.2324, -0.2359,
          -0.2393],
         [ 0.3826, -0.1221, -0.1255, -0.1290, -0.1324,  0.3654,  0.1113,
          -0.1428, -0.1462,  0.3516, -0.1531,  0.0941, -0.1600,  0.0872,
          -0.1669,  0.3309, -0.1738, -0.1772, -0.1807, -0.184

### Updated Loss calculation
Now we'll calculate the updated loss.  Our first pass's loss was 3.5835. Since we're passing through the same example and used a fairly high learning rate we should see a significant improvement with just 1 learning pass. 

In [87]:
loss

tensor(3.5835, grad_fn=<NllLossBackward0>)

In [88]:
y_flat = y.view(-1)
logits_flat = logits.view(-1, logits.size(-1))
updated_loss = F.cross_entropy(logits_flat, y_flat)
print(updated_loss.shape, updated_loss)

torch.Size([]) tensor(3.2931, grad_fn=<NllLossBackward0>)


In [89]:
f'1 round of training resulted in an loss improvment of {loss.item() - updated_loss.item():.4f}'

'1 round of training resulted in an loss improvment of 0.2904'

# SUCCESS!
Our training improved the loss by almost **7%**. There are flaws with this, mainly passing the same example through a second time, but this helps show the fundamentals of what learning does inside a GPT-2 style model. 